# Machine Learning

In [4]:
#import necessary libraries 
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


Read Data

In [5]:
#helper functions
def get_title_from_index(index):
	return moviedataset[moviedataset.index == index]["title"].values[0]

def get_index_from_title(title):
	return moviedataset[moviedataset.title == title]["index"].values[0]
#read data , example file from the video
moviedataset = pd.read_csv("C:\\Users\\Jovan\\Documents\\SC1015-Mini-Project\\cleaned-movie-dataset.csv")


Getting the count matrix

In [6]:
features = ['keywords','casts','genre']
for feature in features:
    moviedataset[feature] = moviedataset[feature].fillna('')

cv = CountVectorizer()

count_matrix = {}

for feature in features:
    count_matrix[feature] = cv.fit_transform(moviedataset[feature])

cossimval = {}

for feature in features:
    cossimval[feature] = cosine_similarity(count_matrix[feature])






KeyError: 'genres'

Time to recommend a movie

In [ ]:
#get a user input on a movie

fav_movie = 'The Fault in Our Stars'

#get index of movie

movieindex = get_index_from_title(fav_movie)

similar_movies = {}

for feature in features:
    similar_movies[feature] =  list(enumerate(cossimval[feature][movieindex]))

sorted_similar_movies = {}

for feature in features:
    sorted_similar_movies[feature] = sorted(similar_movies[feature],key=lambda x:x[1],reverse=True)


## Step 8: Print titles of first 10 movies based off of
i=0
for element in sorted_similar_movies['keywords']:
		print (get_title_from_index(element[0]))
		i=i+1
		if i>10:
			break

The Fault in Our Stars
Winter's Tale
Nerve
Now Is Good
The City of Your Final Destination
The Other Side of Heaven
Heaven is for Real
Beastly
Our Kind of Traitor
Diary of a Wimpy Kid: Rodrick Rules
Hoot
